In this notebook, we look into some weird stuff in our data fetching, such as:

* String vs. bytes in match_stats data 

#### String vs. bytes in match stats data

In [1]:
import os
from tennis_new.fetch.atp_api.defs import (
    API_RESULTS_DIR,
    MATCH_STATS_HEADER,
    MATCH_SCORES_HEADER
)
import pandas as pd

In [ ]:
match_scores_path = os.path.join(
    API_RESULTS_DIR,
    'match_scores',
    'match_scores_2016-2016.csv'
)
match_scores = pd.read_csv(match_scores_path)

In [ ]:
assert match_scores['match_id'].value_counts().max() == 1

In [ ]:
match_scores[[
    'winner_name',
    'loser_name',
    'winner_player_id',
    'loser_player_id',
    'tourney_slug',
    'tourney_round_name',
    'match_score_tiebreaks'
]].head(10)

In [ ]:
[x for x in match_scores.columns if match_scores[x].dtype == 'O']

In [ ]:
match_scores['winner_name'].dtype

#### Look into match stats

In [ ]:
match_stats_path = os.path.join(
    API_RESULTS_DIR,
    'match_stats',
    'match_stats_2018_0.csv'
)
match_stats = pd.read_csv(match_stats_path)

In [ ]:
match_stats.iloc[0]['match_stats_url_suffix']

In [ ]:
match_stats.iloc[0]

#### Tourney Data

In [ ]:
tourney_path = os.path.join(
    API_RESULTS_DIR,
    'tournaments',
    'tournaments_2017-2019.csv'
)
tourneys = pd.read_csv(tourney_path)

In [ ]:
tourneys.iloc[0]

I can probably decode tourney_name and tourney_location in tournaments upon parsing... 

In [ ]:
[x for x in tourneys.columns if tourneys[x].dtype == 'O']

#### Tourneys Combined...

In [ ]:
from tennis_new.fetch.defs import STORED_DATA_PATH

tourneys_combined_path = os.path.join(STORED_DATA_PATH, 'tournaments', 'combined.csv')
tourneys = pd.read_csv(tourneys_combined_path)

In [ ]:
tourneys.iloc[0]

#### Match Scores Combined...

In [2]:
from tennis_new.fetch.defs import STORED_DATA_PATH

ms_combined_path = os.path.join(STORED_DATA_PATH, 'match_scores', 'combined.csv')
match_scores = pd.read_csv(
    ms_combined_path
)

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
match_scores.columns[23]

'match_stats_url_suffix'

In [4]:
match_scores.iloc[0]

tourney_year_id                                                    2016-451
tourney_order                                                             1
tourney_slug                                                           doha
tourney_url_suffix                 /en/scores/archive/doha/451/2016/results
tourney_round_name                                                   Finals
round_order                                                               1
match_order                                                               1
winner_name                                                  Novak Djokovic
winner_player_id                                                       d643
winner_slug                                                  novak-djokovic
loser_name                                                     Rafael Nadal
loser_player_id                                                        n409
loser_slug                                                     rafael-nadal
winner_seed 

In [ ]:
match_scores.shape

#### Read Joined


In [5]:
from tennis_new.fetch.defs import STORED_DATA_PATH

jd = pd.read_csv(os.path.join(STORED_DATA_PATH, 'joined.tsv'), sep='\t')

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13,17,19,33,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
jd.columns[19]

'match_stats_url_suffix'

In [ ]:
jd['tourney_fin_commit'].value_counts(dropna=False)